In [1]:
# 加载模型推理
import os
import json
import torch
from model.CS_tokenizer import Tokenizer
from model.CS_config import Config
from model.CS_model import SequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = Tokenizer(vocab_file='./model/CS_tokenizer.model')

config = Config()
config.vocab_size = tokenizer.vocab_size
config.problem_type = "multi_label_classification"
config.num_labels = 5

model_path = os.path.join('./model/脑卒中严重程度分级/')
model = SequenceClassification.from_pretrained(model_path, config=config)
model.eval()

with open('./data/test.jsonl', 'r', encoding='utf-8') as f:
    eval_data = f.readlines()
# input作为问题, output作为答案
eval_questions = [json.loads(i)['input'] for i in eval_data]
eval_label = [json.loads(i)['output'] for i in eval_data]

model_predictions = []
for i in range(len(eval_questions)):
    model.to(device)
    input_ids = tokenizer.encode(eval_questions[i], padding='max_length', max_length=2048, truncation=True)
    input_ids = torch.tensor([input_ids], dtype=torch.long)
    input_ids = input_ids.to(device)
    with torch.no_grad():
        outputs = model(input_ids)
        pred_label = outputs.logits.argmax(dim=-1).item()
        if pred_label == 0:
            pred_label = '正常'
        elif pred_label == 1:
            pred_label = '轻度'
        elif pred_label == 2:
            pred_label = '中度'
        elif pred_label == 3:
            pred_label = '重度'
        elif pred_label == 4:
            pred_label = '重度以上'
        model_predictions.append(pred_label)
        print(f"question: {eval_questions[i]} \nlabel: {eval_label[i]} \npredict: {pred_label}\n")
    
# 计算准确率
acc = sum([1 for i in range(len(eval_label)) if eval_label[i] == model_predictions[i]]) / len(eval_label)
print(f"Accuracy: {acc:.4f}")

question: 主诉是：突发右侧肢体无力，言语不清。
  现病史是：患者突然感到右侧肢体无力，口齿不清，家人怀疑可能是中风。查体发现右侧肢体活动受限，言语不清，但无意识障碍。
  既往史是：患者无肝炎、结核等病史，也未曾接受过外伤手术或输血。
 
label: 轻度 
predict: 轻度

question: 主诉是：突发剧烈头痛伴右侧肢体无力。
  现病史是：患者突然出现剧烈头痛，伴随右侧肢体无力，家人紧急送往医院就诊。
  既往史是：患者过去健康状况良好，无特殊病史。
 
label: 中度 
predict: 轻度

question: 主诉是：突发言语不清，右侧面部麻木。
  现病史是：患者突然发现自己说话不清晰，右侧面部有麻木感，赶紧就医求诊。
  既往史是：患者以前健康状况良好，无慢性疾病史。
 
label: 轻度 
predict: 轻度

question: 主诉是：右上肢无力伴头晕。
  现病史是：患者感到右上肢无力，头晕眼花，伴有恶心呕吐症状。
  既往史是：患者无慢性病史，生活作息规律。
 
label: 轻度 
predict: 轻度

question: 主诉是：右侧身体活动困难。
  现病史是：患者发现右侧身体活动困难，步态不稳，家人发现患者表情呆滞。
  既往史是：患者曾有高血压病史，但平时服药控制得较好。
 
label: 中度 
predict: 轻度

question: 主诉是：左侧面部麻木伴眩晕。
  现病史是：患者突然感到左侧面部麻木，伴有眩晕感，怀疑是中风。
  既往史是：患者有轻微高血压史，但一直有规律服药。
 
label: 轻度 
predict: 轻度

question: 主诉是：突发右侧肢体无力，言语不清。
  现病史是：患者突然感到右侧肢体无力，口齿不清，家人怀疑可能是中风。查体发现右侧肢体活动受限，言语不清，但无意识障碍。
  既往史是：患者无肝炎、结核等病史，也未曾接受过外伤手术或输血。
 
label: 轻度 
predict: 轻度

question: 主诉是：突发剧烈头痛伴恶心呕吐。
  现病史是：患者突然发作剧烈头痛，伴有恶心呕吐症状，怀疑是中风。
  既往史是：患者无特殊病史，平时健康状况良好。
 
label: 轻度 
predict: 轻度

question: 主诉是：突发右上